In [3]:
%pip install librosa
%pip install sounddevice
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.9.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 9.5 MB/s eta 0:00:00
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 

In [10]:
import sounddevice as sd
import soundfile as sf

def record_memo(filename: str, duration: float = None, samplerate: int = 16000):
    """
    Records audio from the default microphone and writes it to `filename`.
    If duration is None, it records until you hit Ctrl+C.
    """
    channels = 1
    try:
        if duration:
            print(f"Recording for {duration} seconds…")
            data = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=channels)
            sd.wait()
        else:
            print("Recording… press Ctrl+C to stop.")
            # start an infinite stream that we append to a list
            frames = []
            def callback(indata, frames_count, time, status):
                if status:
                    print("⚠️", status)
                frames.append(indata.copy())
            with sd.InputStream(samplerate=samplerate, channels=channels, callback=callback):
                while True:
                    pass
            data = b"".join(frames)
        sf.write(filename, data, samplerate)
        print(f"Saved recording to {filename}")
    except KeyboardInterrupt:
        # if user stopped early without a fixed duration
        if not duration:
            data = b"".join(frames)
            sf.write(filename, data, samplerate)
            print(f"\nStopped and saved recording to {filename}")
        else:
            raise

# Usage examples:
# 1) record a 30-second memo
record_memo("memo.wav", duration=100)

# 2) record until you Ctrl+C
# record_memo("memo.wav")

Recording for 100 seconds…
Saved recording to memo.wav


In [2]:
import librosa
import os

# load audio, resample to 16 kHz mono
y, sr = librosa.load("memo.wav", sr=16000, mono=True)
# optionally split on silence into chunks <30 s
intervals = librosa.effects.split(y, top_db=30)
chunks = [y[start:end] for start, end in intervals]

In [11]:
from openai import OpenAI
import os
import pprint as pp

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

with open("memo.wav", "rb") as f:
    resp = client.audio.transcriptions.create(
        model="whisper-1",
        file=f,
        response_format="text"
    )

transcript = resp
pp.pprint(transcript)


('Dies kommt nach ständiger Rechtsprechung nur dann in Betracht, wenn eine '
 'eindeutige Unbilligkeit von erheblichem Gewicht vorliegt. Eine solche liegt '
 'vor, wenn aufgrund der atypischen Umstände des Einzelfalls, die sich aus dem '
 'groben Maßstab des § 29 Abs. 1 Nr. 1 Gewerbesteuergesetz allgemein ergebende '
 'Unbilligkeit offensichtlich übertroffen wird, z.B. BfH-Urteil vom 25.11.2009 '
 'I R 18 aus 08. Dies ist z.B. dann der Fall, wenn der Steuerpflichtige in der '
 'Betriebsstätte, die nach Art und Umfang von erheblicher Bedeutung ist, auf '
 'Dauer und ausschließlich Leiharbeitnehmer einsetzt, anstatt, wie sonst im '
 'Allgemeinen üblich, die eingesetzten Arbeitskräfte selbst zu beschäftigen '
 'und dadurch der Gemeinde, in der diese Betriebsstätte unterhalten wird, das '
 'Gewerbesteueraufkommen aus dieser Betriebsstätte vollständig entgeht. Eine '
 'Unbilligkeit im Sinne des § 33 Abs. 1 Satz 1 Gewerbesteuergesetz kommt auch '
 'dann in Betracht, wenn durch das Vorhandens